# CSI4108 (Fundamentals of) Cryptography - Assignment 4

## Student Infomation
Name: Jake Wang


## Import Required Libraries

In [1]:
import random
import hashlib
import hmac
import typing
import math

## Question 1: HMAC-SHA-512

### Implementation

In [2]:
def hmac_sha512_hex(message: bytes, key: bytes):
    block_size_bytes = 1024 // 8

    if len(key) > block_size_bytes:
        key = hashlib.sha512(key).digest()
    else:
        key = key.ljust(block_size_bytes, b'\x00')

    opad = b'\x5c' * block_size_bytes
    ipad = b'\x36' * block_size_bytes

    o_key_pad = bytes(x ^ y for x, y in zip(key, opad))
    i_key_pad = bytes(x ^ y for x, y in zip(key, ipad))

    inner_hash = hashlib.sha512(i_key_pad + message).digest()
    outer_hash = hashlib.sha512(o_key_pad + inner_hash).hexdigest()

    return outer_hash

### Verification

In [3]:
message = "This input string is being used to test my own implementation of HMAC-SHA-512."
key = "CSI 4108"

result_implementation = hmac_sha512_hex(message.encode(), key.encode())
print(f"Result from implementation: {result_implementation}")

result_library = hmac.new(key.encode(), message.encode(), hashlib.sha512).hexdigest()
print(f"Result from library: {result_library}")

print(f"Pass: {result_implementation == result_library}")

Result from implementation: 8792d98f5da85e864b68ef24741d56445956d1950741b7de94a24e5d206b229c07e1af13f3d8ec28dc662668c9616cbbbe41d10effdb2cea88bd10f2753a41e3
Result from library: 8792d98f5da85e864b68ef24741d56445956d1950741b7de94a24e5d206b229c07e1af13f3d8ec28dc662668c9616cbbbe41d10effdb2cea88bd10f2753a41e3
Pass: True


## Question 2: DSA

### System Parameters

In [4]:
# Generated with `openssl dsaparam 1024 160`

# -----BEGIN DSA PARAMETERS-----
# MIIBHwKBgQDYzzwF7Ar1t47y4X+wlZBJXLlYxb7LwDI+R0pat914W04SSwTuFjSX
# 6CLD7WpvkuRZMb2o269uK1EmntKdUyPx3LUHC5BY2K6Q1YwRAvVuf0OubnlttloD
# cIEe4gPqwr8mMfaT3plY17NDv/bq5IPEds3katcBl9ukgylGNsI3zwIVAIsWaCuw
# 2OHOI4EiIM4vVD4UjeZJAoGBAM/klaHh2334YCqZn2HHFZZzzLzcbuy4MLscW08n
# LAYHfs5Q6oU+IHQBSuWlFLdntSOQ/sbjprNgfTa1jiVEaFV7QceEi9gYx5kFGO02
# m8rasB40w0iFNuSVyyTXvI4P9VzRRndDGkNrBwtHt3x38C7lCdLJH6cpdjSOak9c
# WDa9
# -----END DSA PARAMETERS-----

p = "00d8cf3c05ec0af5b78ef2e17fb095" \
    "90495cb958c5becbc0323e474a5ab7" \
    "dd785b4e124b04ee163497e822c3ed" \
    "6a6f92e45931bda8dbaf6e2b51269e" \
    "d29d5323f1dcb5070b9058d8ae90d5" \
    "8c1102f56e7f43ae6e796db65a0370" \
    "811ee203eac2bf2631f693de9958d7" \
    "b343bff6eae483c476cde46ad70197" \
    "dba483294636c237cf"

q = "008b16682bb0d8e1ce23812220ce2f" \
    "543e148de649"

g = "00cfe495a1e1db7df8602a999f61c7" \
    "159673ccbcdc6eecb830bb1c5b4f27" \
    "2c06077ece50ea853e2074014ae5a5" \
    "14b767b52390fec6e3a6b3607d36b5" \
    "8e254468557b41c7848bd818c79905" \
    "18ed369bcadab01e34c3488536e495" \
    "cb24d7bc8e0ff55cd14677431a436b" \
    "070b47b77c77f02ee509d2c91fa729" \
    "76348e6a4f5c5836bd"

p = int(p, 16)
q = int(q, 16)
g = int(g, 16)

### Implementation

In [5]:
KeyPairType: typing.TypeAlias = tuple[int, int]
SignatureType: typing.TypeAlias = tuple[int, int]


class DSA:
    def __init__(self, p: int, q: int, g: int):
        self.p = p
        self.q = q
        self.g = g

    def get_key_pair(self) -> KeyPairType:
        private_key = random.randint(1, self.q - 1)
        public_key = pow(self.g, private_key, self.p)
        return private_key, public_key

    def sign(self, message: bytes, private_key: int, k=None) -> SignatureType:
        k_chosen = k is not None
        if not k_chosen:
            k = random.randint(1, self.q - 1)

        while True:
            r = pow(self.g, k, self.p) % self.q
            if r == 0:
                if k_chosen:
                    raise ValueError("r is 0 with chosen k")
                k = random.randint(1, self.q - 1)
                continue
            h = int.from_bytes(hashlib.sha1(message).digest())
            s = (h + private_key * r) * pow(k, -1, self.q) % self.q
            if s == 0:
                if k_chosen:
                    raise ValueError("s is 0 with chosen k")
                k = random.randint(1, self.q - 1)
                continue
            return r, s

    def verify(self, message: bytes, signature: SignatureType, public_key: int):
        r, s = signature
        if not (0 < r < self.q and 0 < s < self.q):
            raise ValueError("r and s should be in (0, q)")

        h = int.from_bytes(hashlib.sha1(message).digest())
        u_1 = h * pow(s, -1, self.q) % self.q
        u_2 = r * pow(s, -1, self.q) % self.q

        return pow(self.g, u_1, self.p) * pow(public_key, u_2, self.p) % self.p % self.q == r

### Keypair Generation

In [6]:
dsa = DSA(p, q, g)
private_key, public_key = dsa.get_key_pair()
print(f"Private key: {hex(private_key)}")
print(f"Public key: {hex(public_key)}")

Private key: 0x13e44fc6c6593f54586d24dda28cf810b27a6a89
Public key: 0x8e67f3f73580164fd016b463e7ef7f1c0265e5fd13aa741a1bb0ae7b14af7a26fd5c6ac6400f2a26bd6c96f0a70c41467b75a783af95a8a222e580c33b29752e7039773dd1ca6440b6332624bdbdee262f66125b25a65794d9964ad65caa2d05a4ecd3097651fa4d29d9e63462717c7a4ca3d7a918442c5a2580a1a25913fab0


### Signing and Verifying

In [7]:
m_1 = 582346829557612
m_1_bytes = m_1.to_bytes(math.ceil(math.log(m_1, 2 ** 8)))

r_1, s_1 = dsa.sign(m_1_bytes, private_key, k=4108)
print(f"r_1: {hex(r_1)}")
print(f"s_1: {hex(s_1)}")

result = dsa.verify(m_1_bytes, (r_1, s_1), public_key)
print(f"Pass: {result}")

r_1: 0x6e40704674b518835be43f10090f2b8c3e67f298
s_1: 0x3484cbccbe1121700a60ceb23655e9c803bb6c85
Pass: True


## Question 3: DSA Attack on $k$-Reusing

In [8]:
m_2 = 8161474912583
m_2_bytes = m_2.to_bytes(math.ceil(math.log(m_2, 2 ** 8)))

r_2, s_2 = dsa.sign(m_2_bytes, private_key, k=4108)
print(f"r_2: {hex(r_2)}")
print(f"s_2: {hex(s_2)}")

r_2: 0x6e40704674b518835be43f10090f2b8c3e67f298
s_2: 0x2ffdd63a6c7aa982cd714c2e4787c265dd40958a


* From the DSA scheme, we know if $k$ is reused:
\begin{align}
r_1 &\equiv r_2 \equiv r \equiv g ^ k \pmod p \pmod q \\
s_1 &\equiv k ^ {-1} (H(m_1) + xr) \pmod q \\
s_2 &\equiv k ^ {-1} (H(m_2) + xr) \pmod q
\end{align}
* From $(2)$, we know:
\begin{align}
ks_1 \equiv H(m_1) + xr \pmod q
\end{align}
* From $(3)$, we know:
\begin{align}
k \equiv s_2 ^ {-1} (H(m_2) + xr) \pmod q
\end{align}
* Plug $(5)$ into $(4)$, we get:
\begin{align}
s_2 ^ {-1} (H(m_2) + xr) \cdot s_1 &\equiv H(m_1) + xr \pmod q \\
s_1 (H(m_2) + xr) &\equiv s_2 (H(m_1) + xr) \pmod q \\
s_1 H(m_2) + s_1xr &\equiv s_2 H(m_1) + s_2xr \pmod q \\
(s_1 - s_2)xr &\equiv s_2 H(m_1) - s_1 H(m_2) \pmod q \\
x &\equiv (s_2 H(m_1) - s_1 H(m_2))(r(s_1 - s_2)) ^ {-1} \pmod q
\end{align}
* $x$, the private key, can then be calculated:

In [9]:
r = r_1
h_1 = int.from_bytes(hashlib.sha1(m_1_bytes).digest())
h_2 = int.from_bytes(hashlib.sha1(m_2_bytes).digest())
x = (s_2 * h_1 - s_1 * h_2) * pow(r * (s_1 - s_2), -1, dsa.q) % dsa.q
print(f"x = {hex(x)}")

x = 0x13e44fc6c6593f54586d24dda28cf810b27a6a89


## Question 4: DSA Cracking

### Cracking Private Key

In [10]:
# https://en.wikipedia.org/wiki/Baby-step_giant-step
def baby_step_giant_step(base, number, modulus):
    m = math.ceil(math.sqrt(modulus))

    # Precompute baby steps
    baby_steps = {}
    baby = 1
    for r in range(m):
        baby_steps[baby] = r
        baby = (baby * base) % modulus

    giant_step = pow(base, m * (modulus - 2), modulus)

    # Search for a match
    giant = number
    for s in range(m):
        if giant in baby_steps:
            return s * m + baby_steps[giant]
        giant = (giant * giant_step) % modulus

    # No match found
    return None

In [11]:
p, q, g = 103687, 1571, 21947
A = 31377
X = baby_step_giant_step(g, A, p)
print(f"Private key: {X}")

Private key: 602


### Signing

In [12]:
D = 510
D_bytes = D.to_bytes(math.ceil(math.log(D, 2 ** 8)))

dsa = DSA(p, q, g)
r, s = dsa.sign(D_bytes, X, k=1105)
print(f"r: {hex(r)}")
print(f"s: {hex(s)}")

r: 0x1b7
s: 0x39b


### Verifying

In [13]:
dsa.verify(D_bytes, (r, s), A)

True